# PART 3
## Normalize data

Scran normalization: since the cell types show intrinsic differences in library sizes, we will running the normalzation on each cell type separately.

- **INPUT:**
    - ```counts_cf_ctf_gf_sf.RData``` 
    - ```anno_cells_cf_ctf_gf_sf.RData```
    - ```anno_samples_cf_ctf_gf_sf.RData```
    - ```anno_genes_cf_ctf_gf_sf.RData```
    - ```cell_relabelling.csv``` file containing unified cell type annotations. Stored in additional_input_files sub-directory.
    
    
- **OUTPUT:**

    -  ```counts_norm.mtx``` 
    -  ```anno_cells_norm.txt``` 
    -  ```anno_samples_norm.txt``` 
    -  ```anno_genes_norm.txt``` 

### load data

In [1]:
library(ggplot2)
library(SingleCellExperiment)
library(scran)
library(scater)
library(Matrix)

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges

In [2]:
# if (!require("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install(c("SingleCellExperiment","scran","scater"))

In [3]:
path_to_additional_files="../additional_input_files/additional_input_files/"
path_in <- "../2.filtering/outs/"
path_out <- "outs/"

In [8]:
# load cell relabelling file
print("load cell_relabelling")
tsv_file="../../cell_relabelling.csv"
cell_relabelling <- read.csv(file = tsv_file
                             ,sep = ";"
                             )

cell_relabelling <- cell_relabelling[cell_relabelling$source == "van Galen et al., 2019" | cell_relabelling$source == "Oetjen et al., 2018", ]
rownames(cell_relabelling) <- cell_relabelling$cell_type_original
print(cell_relabelling)

[1] "load cell_relabelling"
                                       cell_type_original      cell_type
B                                                       B              B
CD10+ B cells                               CD10+ B cells              B
CD20+ B cells                               CD20+ B cells              B
ProB                                                 ProB              B
Mono                                                 Mono           Mono
CD14+ monocytes                           CD14+ monocytes           Mono
CD16+ monocytes                           CD16+ monocytes           Mono
Mono-like                                       Mono-like           Mono
ProMono                                           ProMono           Mono
Monocyte progenitors                 Monocyte progenitors           Mono
ProMono-like                                 ProMono-like           Mono
T                                                       T              T
CD4+ memory T cells    

In [9]:
# load counts
print("load counts")
load(paste0(path_in, "counts_cf_ctf_gf_sf.RData"))
print("dim(counts_cf_ctf_gf_sf)")
print(dim(counts_cf_ctf_gf_sf))

[1] "load counts"
[1] "dim(counts_cf_ctf_gf_sf)"
[1] 12485 74583


In [10]:
# load cell annotation
print("load cell annotation")
load(paste0(path_in, "anno_cells_cf_ctf_gf_sf.RData"))
print("str(anno_cells_cf_ctf_gf_sf)")
print(str(anno_cells_cf_ctf_gf_sf))

[1] "load cell annotation"
[1] "str(anno_cells_cf_ctf_gf_sf)"
'data.frame':	74583 obs. of  33 variables:
 $ cell_ID                       : chr  "AML1012-D0_AAAAAGTTACGT" "AML1012-D0_AAAACACCAATC" "AML1012-D0_AAAATAGCCTTT" "AML1012-D0_AAACATTAAACG" ...
 $ cell_type_original            : chr  "GMP" "GMP-like" "Prog-like" "ProMono-like" ...
 $ sample_ID                     : chr  "AML1012-D0" "AML1012-D0" "AML1012-D0" "AML1012-D0" ...
 $ malignant                     : logi  FALSE TRUE TRUE TRUE TRUE TRUE ...
 $ patient_ID                    : chr  "AML1012" "AML1012" "AML1012" "AML1012" ...
 $ dataset                       : chr  "GSE116256" "GSE116256" "GSE116256" "GSE116256" ...
 $ gender                        : chr  "F" "F" "F" "F" ...
 $ age                           : int  32 32 32 32 32 32 32 32 32 32 ...
 $ health_status                 : chr  "AML" "AML" "AML" "AML" ...
 $ case_or_control               : chr  "case" "case" "case" "case" ...
 $ day                           : in

In [11]:
# load sample annotation
print("load sample annotation")
load(paste0(path_in, "anno_samples_cf_ctf_gf_sf.RData"))
print("str(anno_samples_cf_ctf_gf_sf)")
print(str(anno_samples_cf_ctf_gf_sf))

[1] "load sample annotation"
[1] "str(anno_samples_cf_ctf_gf_sf)"
'data.frame':	33 obs. of  15 variables:
 $ sample_ID                    : chr  "AML1012-D0" "AML210A-D0" "AML328-D0" "AML419A-D0" ...
 $ patient_ID                   : chr  "AML1012" "AML210A" "AML328" "AML419A" ...
 $ dataset                      : chr  "GSE116256" "GSE116256" "GSE116256" "GSE116256" ...
 $ gender                       : chr  "F" "M" "F" "F" ...
 $ age                          : int  32 67 74 54 58 70 26 57 42 52 ...
 $ health_status                : chr  "AML" "AML" "AML" "AML" ...
 $ case_or_control              : chr  "case" "case" "case" "case" ...
 $ day                          : int  0 0 0 0 0 0 0 0 0 NA ...
 $ nr_cells_raw                 : int  1136 748 1094 1189 485 2328 1586 933 3813 108 ...
 $ nr_cells_cf                  : int  951 640 965 1091 399 2196 1410 869 3574 93 ...
 $ nr_cells_cf_ctf              : int  946 603 909 1086 388 2186 1407 868 3504 93 ...
 $ total_counts_raw_cf_ctf_gf   

In [12]:
# load gene annotation
print("load gene annotation")
load(paste0(path_in, "anno_genes_cf_ctf_gf_sf.RData"))
print("str(anno_genes_cf_ctf_gf_sf)")
print(str(anno_genes_cf_ctf_gf_sf))

[1] "load gene annotation"
[1] "str(anno_genes_cf_ctf_gf_sf)"
'data.frame':	12485 obs. of  7 variables:
 $ gene_symbol                          : chr  "A1BG" "A1BG-AS1" "A2M-AS1" "A4GALT" ...
 $ is_in_LR_database                    : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ isLigand                             : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ isReceptor                           : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ sum_pseudobulkCellType_expr          : num  40.76 1.67 2.74 10.86 18.2 ...
 $ sum_pseudobulkCellType_expr_threshold: num  1 1 1 1 1 1 1 1 1 1 ...
 $ sum_pseudobulkCellType_expr_filter   : logi  TRUE TRUE TRUE TRUE TRUE TRUE ...
NULL


In [13]:
# color scheme for celltypes
color_celltype <- cell_relabelling$cell_type_color_hex[!duplicated(cell_relabelling$cell_type)]
names(color_celltype) <- cell_relabelling$cell_type[!duplicated(cell_relabelling$cell_type)]

### process data 

In [14]:
counts_norm <- counts_cf_ctf_gf_sf

In [ ]:
# normalize each cell type separately with scran
for(cluster in unique(anno_cells_cf_ctf_gf_sf$cell_type)){
    print(cluster)
  
    idx_cluster <- anno_cells_cf_ctf_gf_sf$cell_type == cluster

    # subset anno_cells_cf_ctf_gf_sf
    anno_cells_sub <- anno_cells_cf_ctf_gf_sf[idx_cluster,]

    # create SingleCellExperiment object
    print("create SingleCellExperiment object")
    original_sce <- SingleCellExperiment(
    assays = list(counts = as.matrix(counts_cf_ctf_gf_sf[,idx_cluster]))
    ,colData = anno_cells_sub
    )

    # comupte size factors
    print("comupte size factors")
    original_sce <- computeSumFactors(original_sce
                                    #,cluster=cluster 
    )

    anno_cells_cf_ctf_gf_sf$sizeFactor[idx_cluster] <- sizeFactors(original_sce)

    # normalize
    print("scran normalization")
    original_sce <- logNormCounts(original_sce
                                ,log = FALSE
                                )
    counts_norm[,idx_cluster] <- original_sce@assays@data@listData$normcounts
}


### visualize 

In [ ]:
print("construct pseudo bulks for cell types")
for(counts_type in c("counts"
                ,"normcounts")
){
  
  ifelse(counts_type == "counts"
         ,counts <- counts_cf_ctf_gf_sf
         ,counts <- counts_norm)
  
  cell_type_IDs <- unique(anno_cells_cf_ctf_gf_sf$cell_type_ID)
  counts_pseudobulkCellType <- as.data.frame(matrix(,nrow = nrow(counts)
                                                    ,ncol = length(cell_type_IDs)
  )
  )
  i <- 1
  for(cell_type_ID in cell_type_IDs){
    #print(cell_type_ID)
    
    idx_cell_type_ID <- anno_cells_cf_ctf_gf_sf$cell_type_ID == cell_type_ID
    
    if(sum(idx_cell_type_ID) == 0){
      counts_pseudobulkCellType[,i] <- 0
    } else if( sum(idx_cell_type_ID) == 1){
      counts_pseudobulkCellType[,i] <- counts[,idx_cell_type_ID]
    } else {
      counts_pseudobulkCellType[,i] <- rowMeans(counts[,idx_cell_type_ID])
    }
    
    i <- i+1
  }
  colnames(counts_pseudobulkCellType) <- cell_type_IDs
  rownames(counts_pseudobulkCellType) <- rownames(counts)
  print("str(counts_pseudobulkCellType)")
  print(str(counts_pseudobulkCellType))
  
  # visualize expression in pseudo sorted bulk
  print("visualize expression in pseudo-bulk cell type")
  for(genes_type in c("all genes"
                      ,"ligands and receptors")){
    ifelse(genes_type == "all genes"
           ,idx_genes <- rep(T,nrow(anno_genes_cf_ctf_gf_sf))
           ,idx_genes <- anno_genes_cf_ctf_gf_sf$isLigand | anno_genes_cf_ctf_gf_sf$isReceptor
    )
    
    my_sampes <- sub("_.*", "", colnames(counts_pseudobulkCellType))
    df <- data.frame(log10_lib_size = log10(colSums(counts_pseudobulkCellType[idx_genes,]) +1)
                     ,sample_ID = sub(".*_"
                                      ,""
                                      ,colnames(counts_pseudobulkCellType)
                     )
                     ,cell_type = sub("_.*"
                                      ,""
                                      ,colnames(counts_pseudobulkCellType)
                     )
    )
    
    print(ggplot(data = df
                 ,aes(x = sample_ID
                      ,y = log10_lib_size
                      ,color = cell_type
                 )
    ) +
      geom_jitter(height = NULL
                  ,width = 0
                  ,alpha = 0.75) +
      ggtitle(paste0(genes_type, ": "
                    ,counts_type)) +
      coord_flip() +
      ylim(c(0,4.5)) +
      scale_color_manual(values = color_celltype)+
      theme_bw()
    )
    
    
    # visualize expression after gene filter
    print("visualize expression after gene filter")
    df <- data.frame(log10_lib_size = log10(colSums(counts[idx_genes,]) +1)
                     ,sample_ID = anno_cells_cf_ctf_gf_sf$sample_ID
    )
    print(str(df))
    
    print(ggplot(data = df
                 ,aes(x = sample_ID
                      ,y = log10_lib_size
                 )
    ) +
      geom_jitter(height = NULL
                  ,size = 0.1
                  ,alpha = 0.2) +
      geom_violin() + 
      ggtitle(paste("log10 expression"
                    ,counts_type
                    ,genes_type)) +
      coord_flip() +
      ylim(c(0,6))+
      theme_bw()
    )
    rm(df)
    
  }
  
}


In [ ]:
anno_cells_norm <- anno_cells_cf_ctf_gf_sf
anno_genes_norm <- anno_genes_cf_ctf_gf_sf
anno_samples_norm <- anno_samples_cf_ctf_gf_sf

### export

In [ ]:
# export ####
print("export")
print("export counts_norm.mtx")
writeMM(obj = Matrix(as.matrix(counts_norm)
                     , sparse=TRUE)
        , file = paste0(path_out, "counts_norm.mtx")
        )

In [ ]:
# export anno_cells_norm
print("export anno_cells_norm.txt")
write.table(anno_cells_norm
            ,file = paste0(path_out, "anno_cells_norm.txt")
             ,sep = "\t"
            ,row.names = FALSE )

In [ ]:
# export anno_genes
print("export anno_genes_norm.txt")
write.table(anno_genes_norm
            ,file = paste0(path_out, "anno_genes_norm.txt")
            ,sep = "\t"
            ,row.names = FALSE
)

In [ ]:
# export anno_samples
print("export anno_samples_norm.txt")
write.table(anno_samples_norm
            ,file = paste0(path_out, "anno_samples_norm.txt")
            ,sep = "\t"
            ,row.names = FALSE
)